## **Web Site**

https://notaries-directory.eu/en/search

## **Acceso a la Web y Aceptación de Cookies**

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import os
from os import system
import random
import logging
from datetime import datetime

# Configurar logging
logging.basicConfig(
    filename=f'notarios_scraping_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

url_base = "https://notaries-directory.eu/en/search?page=0"

def setup_driver():
    """Configurar y devolver una instancia del navegador Chrome"""
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Descomenta para ejecución sin interfaz gráfica
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    # Configurar el driver de Selenium
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def aceptar_cookies(driver):
    """Aceptar cookies si aparece el popup"""
    try:
        WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="sliding-popup"]/div/div[2]/div/div/button'))
        ).click()
        logging.info("Cookies aceptadas")
    except (TimeoutException, NoSuchElementException):
        logging.info("No se encontró el popup de cookies o ya fue aceptado")

def extraer_datos_notario(driver, notario_actual, indice, total_notarios):
    """Extraer los datos de un notario específico"""
    datos = {}
    
    try:
        # Hacer scroll hasta el notario para asegurarnos que es visible
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", notario_actual)
        time.sleep(0.5)
        
        # Extraer nombre antes de hacer clic
        try:
            nombre = notario_actual.find_element(By.XPATH, './/div/h3').text.strip()
        except:
            nombre = f"Notario #{indice}"
        
        logging.info(f"Procesando notario: {nombre} ({indice}/{total_notarios})")
        print (f"Procesando notario: {nombre} ({indice}/{total_notarios})")
        
        # Encontrar y hacer clic en el botón "VIEW MORE" para este notario
        boton_view_more = notario_actual.find_element(By.XPATH, './/a/span/span')
        driver.execute_script("arguments[0].click();", boton_view_more)
        
        # Esperar a que la página de detalles cargue
        time.sleep(2)
        
        # Extraer información detallada
        try:
            first_name = driver.find_element(By.XPATH, '/html/body/main/div/div[2]/div/div/div/div[1]/div[1]/h1/span[1]').text
        except:
            first_name = "No disponible"
            
        try:
        # Usar un selector más específico basado en la clase
            e_mail = driver.find_element(By.CSS_SELECTOR, '.info-with-icon.link-redirect.mail a').text
            if not e_mail:
                e_mail = driver.find_element(By.CSS_SELECTOR, '.info-with-icon.link-redirect.mail a').get_attribute('href').replace('mailto:', '')
        except:
            try:
        # Alternativa usando XPath y la clase
                e_mail = driver.find_element(By.XPATH, '//div[contains(@class, "info-with-icon") and contains(@class, "link-redirect") and contains(@class, "mail")]/a').text
                if not e_mail:
                    href = driver.find_element(By.XPATH, '//div[contains(@class, "info-with-icon") and contains(@class, "link-redirect") and contains(@class, "mail")]/a').get_attribute('href')
                    e_mail = href.replace('mailto:', '') if href else "No disponible"
            except:
                e_mail = "No disponible"
        
        try:
            country = driver.find_element(By.XPATH,'/html/body/main/div/div[2]/div/div/div/div[1]/div[2]/div/span').text
        except:
            country = "No disponible"
        
               
        # Crear diccionario con los datos
        datos = {
            "Full_name": nombre,
            "First_name": first_name,
            "e-mail": e_mail,
            "Country": country
        }
        
        print (f"✅ Información extraída del notario {indice}")
        print(f"Full_name: {nombre}")
        print(f"First_name: {first_name}")
        print(f"e-mail: {e_mail}")
        print(f"Country: {country}")
        
        
        
        
        # Volver a la lista principal
        try:
            boton_volver = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div[2]/div/div/div/a/span'))
            )
            driver.execute_script("arguments[0].click();", boton_volver)
        except:
            logging.warning("No se pudo encontrar el botón volver, usando navegación del navegador")
            driver.back()
        
        # Esperar a que la lista de notarios cargue nuevamente
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="list-element"]'))
        )
        
        return datos
        
    except Exception as e:
        logging.error(f"Error al extraer datos del notario {indice}: {str(e)}")
        return {
            "Full_name": nombre if 'nombre' in locals() else f"Notario #{indice}",
            "First_name": "Error",
            "e-mail": "Error",
            "Country": "Error",
            "Teléfono": "Error",
            "Error": str(e)
        }

def ir_a_siguiente_pagina(driver, pagina_actual):
    """Navegar a la siguiente página de resultados usando la URL directamente"""
    try:
        # Calcular el número de la siguiente página
        siguiente_pagina = pagina_actual + 1
        
        # Construir la URL con el número de página
        url_siguiente = f"https://notaries-directory.eu/en/search?page={siguiente_pagina}"
        
        # Navegar directamente a esa URL
        logging.info(f"Navegando a la página {siguiente_pagina} usando URL: {url_siguiente}")
        driver.get(url_siguiente)
        
        # Esperar a que la nueva página cargue
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//li[@class="list-element"]'))
        )
        time.sleep(2)  # Espera adicional para asegurar la carga completa
        return True
    except Exception as e:
        logging.error(f"Error al navegar a la siguiente página: {str(e)}")
        return False

def guardar_datos_parciales(datos, pagina_actual):
    """Guardar los datos recolectados hasta el momento"""
    if datos:
        df = pd.DataFrame(datos)
        archivo = f"datos_notarios_hasta_pagina_{pagina_actual}.csv"
        df.to_csv(archivo, index=False)
        logging.info(f"Datos guardados en {archivo}: {len(datos)} registros")

def obtener_numero_pagina_actual(driver):
    """Obtener el número de la página actual de resultados"""
    try:
        pagina_actual = driver.find_element(By.XPATH, '//li[@class="pager__item is-active"]/a').text
        return int(pagina_actual)
    except:
        return None

def extraer_todos_los_notarios():
    """Función principal para escrapear todas las páginas"""
    driver = setup_driver()
    driver.get(url_base)
    time.sleep(5)
    
    aceptar_cookies(driver)
    
    datos_notarios = []
    pagina_actual = 0
    pagina_final = 7068  # Número total de páginas
    
    # Verificar si existe un archivo de estado para continuar desde donde se quedó
    archivo_estado = "estado_scraping.txt"
    if os.path.exists(archivo_estado):
        with open(archivo_estado, "r") as f:
            ultima_pagina = int(f.read().strip())
            pagina_actual = ultima_pagina
            logging.info(f"Continuando desde la página {pagina_actual}")
            
            # Cargar datos existentes si existen
            archivo_datos = f"datos_notarios_hasta_pagina_{pagina_actual-1}.csv"
            if os.path.exists(archivo_datos):
                datos_existentes = pd.read_csv(archivo_datos)
                datos_notarios = datos_existentes.to_dict('records')
                logging.info(f"Cargados {len(datos_notarios)} registros existentes")
            
            # Navegar a la página donde se quedó
            if pagina_actual > 1:
                # Aquí necesitarías implementar la navegación directa a la página específica
                # Esto depende de cómo está estructurada la URL o la paginación del sitio
                # Por ejemplo, si la URL tiene un parámetro de página:
                driver.get(f"{url_base}?page={pagina_actual}")
    
    try:
        while pagina_actual <= pagina_final:
            logging.info(f"Procesando página {pagina_actual} de {pagina_final}")
            
            # Esperar a que los elementos de la lista estén disponibles
            try:
                notarios = WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//li[@class="list-element"]'))
                )
                total_notarios = len(notarios)
                logging.info(f"Se encontraron {total_notarios} notarios en la página {pagina_actual}")
                
                # Procesar cada notario en la página actual
                for i in range(total_notarios):
                    # Obtener la lista actualizada para evitar StaleElementReferenceException
                    notarios_actualizados = WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.XPATH, '//li[@class="list-element"]'))
                    )
                    
                    if i < len(notarios_actualizados):
                        notario_actual = notarios_actualizados[i]
                        datos_notario = extraer_datos_notario(driver, notario_actual, i+1, total_notarios)
                        if datos_notario:
                            datos_notarios.append(datos_notario)
                        
                        # Pausa aleatoria para evitar ser detectado como bot
                        time.sleep(random.uniform(0.5, 1.5))
                
                # Guardado periódico (cada página)
                guardar_datos_parciales(datos_notarios, pagina_actual)
                
                # Guardar estado actual
                with open(archivo_estado, "w") as f:
                    f.write(str(pagina_actual))
                
                # Pasar a la siguiente página si no es la última
                if pagina_actual < pagina_final:
                    
                    exito = ir_a_siguiente_pagina(driver, pagina_actual)
                    if exito:
                        pagina_actual += 1
                    else:
                        logging.error(f"No se pudo avanzar a la página {pagina_actual + 1}. Reintentando...")
                        driver.refresh()
                        time.sleep(5)
                        # Si después de varios intentos no funciona, podríamos implementar un contador de reintentos
                else:
                    break
                
            except Exception as e:
                logging.error(f"Error procesando la página {pagina_actual}: {str(e)}")
                driver.save_screenshot(f"error_pagina_{pagina_actual}.png")
                # Intentar refrescar y continuar
                driver.refresh()
                time.sleep(5)
    
    except KeyboardInterrupt:
        logging.info("Proceso interrumpido manualmente. Se guardarán los datos recolectados hasta ahora.")
    
    except Exception as e:
        logging.error(f"Error inesperado: {str(e)}")
        driver.save_screenshot(f"error_inesperado_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png")
    
    finally:
        # Guardar todos los datos recolectados
        if datos_notarios:
            df_final = pd.DataFrame(datos_notarios)
            archivo_final = "datos_notarios_completo.csv"
            df_final.to_csv(archivo_final, index=False)
            logging.info(f"Datos completos guardados en {archivo_final}: {len(datos_notarios)} registros")
        
        driver.quit()
        return datos_notarios

if __name__ == "__main__":
    inicio = time.time()
    resultados = extraer_todos_los_notarios()
    fin = time.time()
    duracion = (fin - inicio) / 60  # Duración en minutos
    
    logging.info(f"✅Proceso completado en {duracion:.2f} minutos")
    logging.info(f"Se extrajeron datos de {len(resultados)} notarios en total")

Procesando notario: David METAY (1/7)
✅ Información extraída del notario 1
Full_name: David METAY
First_name: David
e-mail: david.metay@notaires.fr
Country: 12 PLACE DE L HOTEL DE VILLE
SAINT-ÉTIENNE, France
Procesando notario: Camille COFFIN (2/7)
✅ Información extraída del notario 2
Full_name: Camille COFFIN
First_name: Camille
e-mail: c.coffin@1587.notaires.fr
Country: 30 BOULEVARD CHARLES DE GAULLE
SANNOIS, France
Procesando notario: KRYSTIAN JÓZEFIAK (3/7)
✅ Información extraída del notario 3
Full_name: KRYSTIAN JÓZEFIAK
First_name: KRYSTIAN
e-mail: kancelaria@notariusz-kj.pl
Country: MARCINA KASPRZAKA 26/4
60-237 POZNAŃ, POZNAŃ, POLAND
Procesando notario: Roxane DESGRANGES-BROT (4/7)
✅ Información extraída del notario 4
Full_name: Roxane DESGRANGES-BROT
First_name: Roxane
e-mail: roxane.desgranges.97107@notaires.fr
Country: RUE FERDINAND FOREST
BAIE-MAHAULT, France
Procesando notario: Nathalie DERMIENCE (5/7)
✅ Información extraída del notario 5
Full_name: Nathalie DERMIENCE
Firs